In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
import pandas
from mino_repo.minorepo import MinoRepo

ImportError: No module named 'mino_repo.minorepo'

In [ ]:
zfile = '../../forecasting/data/full_data_with_weather'
partial = True
zcols = [1,2,3,4,5,6,7]
if partial:
    _nrows = 10**6
    panda_readed = pandas.read_csv(zfile, usecols = zcols , nrows= _nrows)
else:
    panda_readed = pandas.read_csv(zfile, usecols = zcols)
bb = MinoRepo(panda_readed)

In [ ]:
del panda_readed
_fields_to_move = ['humidity','temperature','pressure','wind_speed']
dim_table_name = 'dim_weather'
bb.create_dim(_fields_to_move, dim_table_name)

In [ ]:
_fields_to_move = 'date'
dim_table_name = 'dim_time'
bb.create_dim(_fields_to_move, dim_table_name)

In [ ]:
bb.master_tables['dim_time']['date'] = pandas.to_datetime(bb.master_tables['dim_time']['date'], format='%Y-%m-%d %H:%M:%S.%f')
bb.master_tables['dim_time']['rDate'] = bb.master_tables['dim_time']['date'].map(lambda x: x.date())
bb.master_tables['dim_time']['hour'] = bb.master_tables['dim_time']['date'].map(lambda x: x.hour)

In [ ]:
bb.create_dim('cups','dim_cups')

In [ ]:
bb.reshape_dims()

In [ ]:
bb.filter_facts('2015-09-28','date')

In [ ]:
bb.dim_fields